In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.4'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,173 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,514 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-sec

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# Read in data from S3 Bucket
from pyspark import SparkFiles
url = "https://project4-nyctaxi.s3.us-east-1.amazonaws.com/new_train.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("new_train.csv"), sep=",", header=True)
df.show()

+-------------------+-------------------+---------------+---------------+----------------+------------------+-------------+------------------+------------------+------------------+-----------------+
|    pickup_datetime|   dropoff_datetime|passenger_count|pickup_location|dropoff_location|store_and_fwd_flag|trip_duration|   pickup_latitude|  pickup_longitude|  dropoff_latitude|dropoff_longitude|
+-------------------+-------------------+---------------+---------------+----------------+------------------+-------------+------------------+------------------+------------------+-----------------+
|2024-01-03 22:50:16|2024-01-03 22:59:25|            1.0|             50|             236|                 N|          549|218437.82028573466| 985597.5343919232| 223613.7535760244|996155.3867656847|
|2024-01-25 09:33:05|2024-01-25 09:48:58|            1.0|            238|             230|                 N|          953|227717.51805898765| 991712.7644413216| 216099.1397683868|988628.0600467023|
|2024